In [0]:
import pickle
with open('/content/drive/My Drive/Colab Notebooks/corpus.pkl','rb') as f:
    corpus=pickle.load(f)

In [3]:

import pandas as pd
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/raw_data_back_model.csv",sep="\t")
data = data[['overall', 'ordinal' , 'reviewText','label']]
data.head()
y=data.iloc[:,3]

df_frequency_map={1:1,-1:0}
data.label = data.label.map(df_frequency_map)
data.head()

,overall,ordinal,reviewText,label
0,3.0,735454,Ate lunch here today for the first time. Food ...,0
1,4.0,735423,This place was kinda hard to find. But after w...,0
2,5.0,734716,"For the summer, my brother is lucky enough to ...",0
3,5.0,733451,Randomly stopped in here last night (Thursday)...,0
4,4.0,733696,YUM this place is so goooood. I only recently ...,1


In [0]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['reviewText'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,ngram_range=(1, 3))
x = cv.fit_transform(corpus)
df1 = pd.DataFrame(x.toarray(), columns=cv.get_feature_names())

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=10000,ngram_range=(1,1))
x = cv.fit_transform(corpus)
df1 = pd.DataFrame(x.toarray(), columns=cv.get_feature_names())

In [0]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df1)
scaled_data = scaler.transform(df1)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(random_state=25)
pca.fit(df1)
x_pca = pca.transform(df1)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.20, random_state = 25)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 25)

In [9]:
auth_series = pd.DataFrame({'label':y_test}) 
Spam = auth_series[auth_series['label']==1]

NotSpam = auth_series[auth_series['label']==0]
print(Spam.shape,NotSpam.shape)

(775, 1) (9611, 1)


In [0]:
# Training model using Naive bayes classifier

from sklearn.model_selection import train_test_split
X_train_, X_test_, y_train_, y_test_ = train_test_split(df1, y, test_size = 0.20, random_state = 25)
#X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train_, y_train_, test_size = 0.25, random_state = 25)

from sklearn.naive_bayes import MultinomialNB
classifier_naive = MultinomialNB().fit(X_train_, y_train_)

y_pred_naive=classifier_naive.predict(X_test_)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_naive = confusion_matrix(y_test, y_pred_naive)
bl_naive=brier_score_loss(y_test, y_pred_naive)
ro_naive=roc_auc_score(y_test, y_pred_naive)
acc_naive=accuracy_score(y_test,y_pred_naive)
print(cm_naive)
print(round(bl_naive,2))
print(round(ro_naive,2))
print(round(acc_naive,2))
print(classification_report(y_test,y_pred_naive))

In [0]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
classifier=xgboost.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train,y_train)


In [0]:
random_search.best_estimator_

In [0]:
import xgboost
classifier_xg = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
              learning_rate=0.3, max_delta_step=0, max_depth=15,
              min_child_weight=3, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
classifier_xg.fit(X_train, y_train)

# Predicting the Test set results
y_pred_xg = classifier_xg.predict(X_test)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_xg = confusion_matrix(y_test, y_pred_xg)
bl_xg=brier_score_loss(y_test, y_pred_xg)
ro_xg=roc_auc_score(y_test, y_pred_xg)
acc_xg=accuracy_score(y_test,y_pred_xg)
print(cm_xg)
print(round(bl_xg,2))
print(round(ro_xg,2))
print(round(acc_xg,2))
print(classification_report(y_test,y_pred_xg))

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200,300,400,500],
              'max_features':randint(1,21),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,4),
              }
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=9)
    #CV = Cross-Validation ( here using Stratified KFold CV)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 40, X_train, y_train)

In [0]:
rf_parameters

In [5]:
from sklearn.ensemble import RandomForestClassifier
claasifier_rf=RandomForestClassifier(n_jobs=-1, n_estimators=400,bootstrap= True,criterion='entropy',max_depth=None,max_features=12,min_samples_leaf= 1)
claasifier_rf.fit(df1, y)
'''y_pred_rf = claasifier_rf.predict(X_test)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_rf = confusion_matrix(y_test, y_pred_rf)
bl_rf=brier_score_loss(y_test, y_pred_rf)
ro_rf=roc_auc_score(y_test, y_pred_rf)
acc_rf=accuracy_score(y_test,y_pred_rf)
print(cm_rf)
print(round(bl_rf,2))
print(round(ro_rf,2))
print(round(acc_rf,2))
print(classification_report(y_test,y_pred_rf))'''




'y_pred_rf = claasifier_rf.predict(X_test)\n\nfrom sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report\n\ncm_rf = confusion_matrix(y_test, y_pred_rf)\nbl_rf=brier_score_loss(y_test, y_pred_rf)\nro_rf=roc_auc_score(y_test, y_pred_rf)\nacc_rf=accuracy_score(y_test,y_pred_rf)\nprint(cm_rf)\nprint(round(bl_rf,2))\nprint(round(ro_rf,2))\nprint(round(acc_rf,2))\nprint(classification_report(y_test,y_pred_rf))'

In [7]:
#testing
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
Text = input("Please enter The text: ")
data = {'comment_text':[Text]}
feature_vector = pd.DataFrame(data)
print("Feature Vector before encoding:" )
print(feature_vector)

lemmatizer = WordNetLemmatizer()
corpus = []
review = re.sub('[^a-zA-Z]', ' ', feature_vector['comment_text'][0])
review = review.lower()
review = review.split()
    
review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
corpus.append(review)
feature_vectors_enc = cv.transform(corpus)
df2=pd.DataFrame(feature_vectors_enc.toarray(),columns=cv.get_feature_names())
print(df2)
result=claasifier_rf.predict(df2)
print(result)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Please enter The text: good pizza.
Feature Vector before encoding:
  comment_text
0  good pizza.
   aaa  aback  abandoned  abbaye  ...  zoo  zucchini  zuppa  zutto
0  0.0    0.0        0.0     0.0  ...  0.0       0.0    0.0    0.0

[1 rows x 10000 columns]
[0]


In [0]:
import pickle
with open('random_forest.pkl','wb') as f:
    pickle.dump(claasifier_rf,f)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.optimizers import adagrad



def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    optimizer = adagrad(lr=0.01)
    model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)


layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

def create_model():
  # Initialising the ANN
  classifier_ann = Sequential()

  # Adding the input layer and the first hidden layer
  classifier_ann.add(Dense(output_dim = 40,activation='sigmoid',input_dim = 20))
  classifier_ann.add(Dropout(0.3))

  # Adding the second hidden layer
  classifier_ann.add(Dense(output_dim = 20,activation='sigmoid'))
  classifier_ann.add(Dropout(0.3))
  # Adding the output layer
  classifier_ann.add(Dense(output_dim = 1, activation = 'sigmoid'))

  # Compiling the ANN
  optimizer = RMSprop(lr=0.01)
  classifier_ann.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return classifier_ann

classifier_ann=KerasClassifier(build_fn=create_model,validation_split=0.33, epochs=2, batch_size=128)
# Fitting the ANN to the Training set
model_history=classifier_ann.fit(X_train, y_train)

# list all data in history
print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred_ann = classifier_ann.predict(X_test)
y_pred_ann = (y_pred_ann > 0.5)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_ann = confusion_matrix(y_test, y_pred_ann)
bl_ann=brier_score_loss(y_test, y_pred_ann)
ro_ann=roc_auc_score(y_test, y_pred_ann)
acc_ann=accuracy_score(y_test,y_pred_ann)
print(cm_ann)
print(round(bl_ann,2))
print(round(ro_ann,2))
print(round(acc_ann,2))
print(classification_report(y_test,y_pred_ann))

In [0]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(X_train)
testing_set_scaled=sc.transform(X_test)

x_train = np.reshape(training_set_scaled, (training_set_scaled.shape[0], training_set_scaled.shape[1], 1))

x_test = np.reshape(testing_set_scaled, (testing_set_scaled.shape[0], testing_set_scaled.shape[1], 1))

#validation_set_scaled=sc.transform(X_val)

#x_val = np.reshape(validation_set_scaled, (validation_set_scaled.shape[0], validation_set_scaled.shape[1], 1))

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

# Initialising the RNN
def create_model():

  classifier_lstm = Sequential()

  # Adding the first LSTM layer and some Dropout regularisation
  classifier_lstm.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
  classifier_lstm.add(Dropout(0.2))

  # Adding a second LSTM layer and some Dropout regularisation
  classifier_lstm.add(LSTM(units = 50, return_sequences = True))
  classifier_lstm.add(Dropout(0.2))

  # Adding a third LSTM layer and some Dropout regularisation
  classifier_lstm.add(LSTM(units = 50, return_sequences = True))
  classifier_lstm.add(Dropout(0.2))

  # Adding a fourth LSTM layer and some Dropout regularisation
  classifier_lstm.add(LSTM(units = 50))
  classifier_lstm.add(Dropout(0.2))

  # Adding the output layer
  classifier_lstm.add(Dense(units = 1))

  # Compiling the RNN
  classifier_lstm.compile(optimizer = 'RMSprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return classifier_lstm

classifier_lstm=KerasClassifier(build_fn=create_model,validation_split=0.33,epochs=1, batch_size=128)

# Fitting the RNN to the Training set
model_history=classifier_lstm.fit(x_train, y_train)

# list all data in history
print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predicting the Test set results
y_pred_lstm = classifier_lstm.predict(x_test)
y_pred_lstm = (y_pred_lstm > 0.5)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_lstm = confusion_matrix(y_test, y_pred_lstm)
bl_lstm=brier_score_loss(y_test, y_pred_lstm)
ro_lstm=roc_auc_score(y_test, y_pred_lstm)
acc_lstm=accuracy_score(y_test,y_pred_lstm)
print(cm_lstm)
print(round(bl_lstm,2))
print(round(ro_lstm,2))
print(round(acc_lstm,2))
print(classification_report(y_test,y_pred_lstm))

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
classifier = SVC()
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

grid_search.best_params_

In [0]:
classifier = SVC(kernel = 'rbf', gamma=0.7)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score

cross_val=cross_val_score(classifier,x_pca,y,cv=10,scoring='accuracy').mean()

print(cross_val)

print(cm)


In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

error_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [0]:
from sklearn.neighbors import KNeighborsClassifier
# NOW WITH K=2
classifier_knn = KNeighborsClassifier(n_neighbors=2)

classifier_knn.fit(X_train,y_train)
y_pred_knn = classifier_knn.predict(X_test)

from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,brier_score_loss,classification_report

cm_knn = confusion_matrix(y_test, y_pred_knn)
bl_knn=brier_score_loss(y_test, y_pred_knn)
ro_knn=roc_auc_score(y_test, y_pred_knn)
acc_knn=accuracy_score(y_test,y_pred_knn)
print(cm_knn)
print(round(bl_knn,2))
print(round(ro_knn,2))
print(round(acc_knn,2))
print(classification_report(y_test,y_pred_knn))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import seaborn as sns
from google.colab import files
fig=plt.figure(figsize=(12,7))

plt.suptitle("Confusion Matrixes with ngram range(1,3) using k=7,a=10",fontsize=24)
plt.subplots_adjust(wspace = 0.4, hspace= 0.4)

plt.subplot(2,3,1)
plt.title("Xtreme Gradient Boosting")
sns.heatmap(cm_xg,annot=True,cmap="Greys",fmt="d",cbar=False)

plt.subplot(2,3,2)
plt.title("K Nearest Neighbors")
sns.heatmap(cm_knn,annot=True,cmap="Purples",fmt="d",cbar=False)

plt.subplot(2,3,3)
plt.title("Artificial Neural Network")
sns.heatmap(cm_ann,annot=True,cmap="Reds",fmt="d",cbar=False)

plt.subplot(2,3,4)
plt.title("Naive Bayes   ")
sns.heatmap(cm_naive,annot=True,cmap="Greens",fmt="d",cbar=False)

plt.subplot(2,3,5)
plt.title("Long Short Term Memory")
sns.heatmap(cm_lstm,annot=True,cmap="Oranges",fmt="d",cbar=False)

plt.subplot(2,3,6)
plt.title("Random Forest")
sns.heatmap(cm_rf,annot=True,cmap="Blues",fmt="d",cbar=False)
fig.savefig('Confusion Matrixes with ngram range(1,1) Using Count Vectroizer.png', dpi=fig.dpi)
files.download('Confusion Matrixes with ngram range(1,1) Using Count Vectroizer.png') 

plt.show()


In [0]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from google.colab import files

y_pred_ann_ = classifier_ann.predict_proba(X_test)[:,1]
fpr_ann, tpr_ann, thresholds_ann = roc_curve(y_test, y_pred_ann_)
auc_ann = auc(fpr_ann, tpr_ann)

y_pred_lstm_ = classifier_lstm.predict_proba(x_test)[:,1]
fpr_lstm, tpr_lstm, thresholds_lstm = roc_curve(y_test, y_pred_lstm_)
auc_lstm = auc(fpr_lstm, tpr_lstm)

'''y_pred_ann = classifier_ann.predict(X_test).ravel()
fpr_ann, tpr_ann, thresholds_ann = roc_curve(y_test, y_pred_ann_)
auc_ann = auc(fpr_ann, tpr_ann)

y_pred_lstm = classifier_lstm.predict(x_test).ravel()
fpr_lstm, tpr_lstm, thresholds_lstm = roc_curve(y_test, y_pred_lstm)
auc_lstm = auc(fpr_lstm, tpr_lstm)'''

y_pred_rf_ = claasifier_rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf_)
auc_rf = auc(fpr_rf, tpr_rf)

y_pred_xg_ = classifier_xg.predict_proba(X_test)[:, 1]
fpr_xg, tpr_xg, thresholds_xg = roc_curve(y_test, y_pred_xg_)
auc_xg = auc(fpr_xg, tpr_xg)

y_pred_naive_ = classifier_naive.predict_proba(X_test_)[:, 1]
fpr_naive, tpr_naive, thresholds_naive = roc_curve(y_test_, y_pred_naive_)
auc_naive = auc(fpr_naive, tpr_naive)

y_pred_knn_ = classifier_knn.predict_proba(X_test)[:, 1]
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_pred_knn_)
auc_knn = auc(fpr_knn, tpr_knn)

fig=plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_ann, tpr_ann, label='ANN (area = {:.3f})'.format(auc_ann))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.plot(fpr_lstm, tpr_lstm, label='LSTM (area = {:.3f})'.format(auc_lstm))
plt.plot(fpr_naive, tpr_naive, label='NB (area = {:.3f})'.format(auc_naive))
plt.plot(fpr_xg, tpr_xg, label='XGB (area = {:.3f})'.format(auc_xg))
plt.plot(fpr_knn, tpr_knn, label='KNN (area = {:.3f})'.format(auc_knn))
plt.xlabel('False positive rate (Specificity)')
plt.ylabel('True positive rate (Sensitivity)')
plt.title('ROC curve with ngram range(1,3) using k=7,a=10')
plt.legend(loc='best')
fig.savefig('ROC curve with ngram range(1,2).png', dpi=fig.dpi)
files.download('ROC curve with ngram range(1,2).png') 
plt.show()

In [0]:
import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve
from google.colab import files

# #############################################################################
# Plot calibration plots

fig=plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))

ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
for clf, name in [(classifier_xg, 'Xg Boost'),
                  (classifier_naive, 'Naive Bayes'),
                  (classifier_ann, 'K Nearest Negihbors'),
                  (claasifier_rf, 'Random Forest'),
                  (classifier_ann,'Artifical Neural Network'),
                  (classifier_lstm,'LSTM')
                  ]:
    if hasattr(clf, "predict_proba"):
        if name=='Naive Bayes':
          prob_pos = clf.predict_proba(X_test_)[:, 1]
        elif name=='LSTM':
          prob_pos = clf.predict_proba(x_test)[:, 1]
          prob_pos=np.interp(a, (a.min(), a.max()), (0, +1))
        else:
          prob_pos = clf.predict_proba(X_test)[:, 1]
    else:  # use decision function
        prob_pos = clf.decision_function(X_test)
        prob_pos = \
            (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
    fraction_of_positives, mean_predicted_value = \
        calibration_curve(y_test, prob_pos, n_bins=10)

    ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
             label="%s" % (name, ))

    ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
             histtype="step", lw=2)

ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="lower right")
ax1.set_title('Calibration plots (reliability curve) with ngram range(1,1) using k=7,a=10')

ax2.set_xlabel("Mean predicted value")
ax2.set_ylabel("Count")
ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
fig.savefig('Calibration curve with ngram range(1,1).png', dpi=fig.dpi)
files.download('Calibration curve with ngram range(1,1).png') 
plt.show()

In [0]:
from sklearn.utils.validation import check_is_fitted
from sklearn.calibration import (CalibratedClassifierCV,
                                 _CalibratedClassifier,
                                 _SigmoidCalibration)
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, label_binarize
from sklearn.isotonic import IsotonicRegression
from google.colab import files
from sklearn.calibration import calibration_curve

def compute_score(model, X, y, verbose=True, round_digits=3):
    import sklearn.metrics as metrics
    y_score = model.predict_proba(X)[:, 1]

    auc = round(metrics.roc_auc_score(y, y_score), round_digits)
    log_loss = round(metrics.log_loss(y, y_score), round_digits)
    brier = round(metrics.brier_score_loss(y, y_score), round_digits)

    precision, recall, threshold = metrics.precision_recall_curve(y, y_score)
    f1 = 2 * (precision * recall) / (precision + recall)

    mask = ~np.isnan(f1)
    f1 = f1[mask]
    precision = precision[mask]
    recall = recall[mask]

    best_index = np.argmax(f1)
    precision = round(precision[best_index], round_digits)
    recall = round(recall[best_index], round_digits)
    f1 = round(f1[best_index], round_digits)

    if verbose:
        print('auc: ', auc)
        print('precision: ', precision)
        print('recall: ', recall)
        print('f1: ', f1)
        print('brier: ', brier)
        print('log_loss: ', log_loss)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'log_loss': log_loss,
        'brier': brier
    }

class CalibratedPrefitClassifier(CalibratedClassifierCV):

    def __init__(self, base_estimator, method):
        super().__init__(base_estimator, method, cv='prefit')

    def fit(self, X, y, sample_weight=None):
        """
        Fit the calibrated model

        Parameters
        ----------
        X : pd.DataFrame, shape (n_samples, n_features)
            Training data.

        y : array-like, shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.

        Returns
        -------
        self : object
            Returns an instance of self.
        """
        # scikit-learn has a check_X_y here that will convert our dataframe
        # to a numpy array, remove that part to prevent the conversion

        le = LabelBinarizer().fit(y)
        self.classes_ = le.classes_

        # the main difference from scikit-learn is that our cv parameter will
        # always be 'prefit', hence we don't need to handle the cross validation case,
        # we still honor that original attribute list type, even though it isn't necessary
        self.calibrated_classifiers_ = []
        calibrated_classifier = CalibratedClassifier(self.base_estimator, method=self.method)
        if sample_weight is not None:
            calibrated_classifier.fit(X, y, sample_weight)
        else:
            calibrated_classifier.fit(X, y)

        self.calibrated_classifiers_.append(calibrated_classifier)
        return self

    def predict_proba(self, X):
        """
        Posterior probabilities of classification

        This function returns posterior probabilities of classification
        according to each class on an array of test vectors X.

        Parameters
        ----------
        X : pd.DataFrame, shape (n_samples, n_features)
            The samples.

        Returns
        -------
        proba : array, shape (n_samples, n_classes)
            The predicted probas.
        """
        # the check_is_fitted method might change to not needing to pass
        # the attribute that we're checking for in the future
        check_is_fitted(self, ['classes_', 'calibrated_classifiers_'])

        # Compute the arithmetic mean of the predictions of the calibrated
        # classifiers
        mean_proba = np.zeros((X.shape[0], len(self.classes_)))
        for calibrated_classifier in self.calibrated_classifiers_:
            proba = calibrated_classifier.predict_proba(X)
            mean_proba += proba

        mean_proba /= len(self.calibrated_classifiers_)
        return mean_proba


class CalibratedClassifier(_CalibratedClassifier):

    def fit(self, X, y, sample_weight=None):
        """
        Calibrate the fitted model

        Parameters
        ----------
        X : pd.DataFrame, shape (n_samples, n_features)
            Training data.

        y : array-like, shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.

        Returns
        -------
        self : object
            Returns an instance of self.
        """
        self.label_encoder_ = LabelEncoder()
        if self.classes is None:
            self.label_encoder_.fit(y)
        else:
            self.label_encoder_.fit(self.classes)

        self.classes_ = self.label_encoder_.classes_
        Y = label_binarize(y, self.classes_)

        # df is the predicted score/probability of the base_estimator
        df, idx_pos_class = self._preproc(X)

        self.calibrators_ = []
        for k, this_df in zip(idx_pos_class, df.T):
            if self.method == 'isotonic':
                # coerce the predicted score to a double to prevent type error
                this_df = this_df.astype(np.float64)
                calibrator = IsotonicRegression(out_of_bounds='clip')
            elif self.method == 'sigmoid':
                calibrator = _SigmoidCalibration()
            else:
                raise ValueError('method should be "sigmoid" or '
                                 '"isotonic". Got %s.' % self.method)
            calibrator.fit(this_df, Y[:, k], sample_weight)
            self.calibrators_.append(calibrator)

        return self

def plot_calibration_curve(estimators_names, X, y):

    fig = plt.figure(figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    estimator_metrics = []
    for estimator, name in estimators_names:
        proba = estimator.predict_proba(X)[:, 1]
        #proba=np.interp(proba, (proba.min(), proba.max()), (0, +1))
        prob_true, prob_pred = calibration_curve(y, proba, n_bins=10)


        #metric_dict = compute_score(estimator, X, y, verbose=False)
        #metric_dict['name'] = name
        #estimator_metrics.append(metric_dict)

        ax1.plot(prob_pred, prob_true, 's-',label=name)
        ax2.hist(proba, range=(0, 1), bins=10, label=name, histtype='step', lw=2)

    ax1.plot([0, 1], [0, 1], 'k:')

    ax1.set_xlabel('Predicted Probability')
    ax1.set_ylabel('True Probability')
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc='lower right')
    ax1.set_title('Calibration Plots (Reliability Curve)')

    #Naive Bayes
    #Random Forest
    #Xtreme Gradient Boosting
    #Artifical Neural Network
    #Long Short Term Memory
    #K Nearest Neighbors
    ax2.set_xlabel('Predicted scores')
    ax2.set_ylabel('Count')
    ax1.set_title('Calibration plot (reliability curve) of K Nearest Neighbors with ngram range(1,3) using k=7,a=10')
    ax2.legend(loc='upper center', ncol=2)

    plt.tight_layout()
    #df_metrics = pd.DataFrame(estimator_metrics)
    return fig

xgb_sigmoid = CalibratedPrefitClassifier(classifier_knn, method='sigmoid')
xgb_sigmoid.fit(X_val, y_val)

xgb_isotonic = CalibratedPrefitClassifier(classifier_knn, method='isotonic')
xgb_isotonic.fit(X_val, y_val)
estimators_names = [
    (classifier_knn, 'Uncalibrated'),
    (xgb_sigmoid, 'Calibrated(Platt)'),
    (xgb_isotonic, 'Calibrated(Isotonic)')
]
fig = plot_calibration_curve(estimators_names, X_test, y_test)
fig.savefig('Calibration.png', dpi=fig.dpi)
files.download('Calibration.png')

In [0]:
3# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from google.colab import files

ann_precision, ann_recall, _ = precision_recall_curve(y_test, y_pred_ann_)
lstm_precision, lstm_recall, _ = precision_recall_curve(y_test, y_pred_lstm_)
knn_precision, knn_recall, _ = precision_recall_curve(y_test, y_pred_knn_)
nb_precision, nb_recall, _ = precision_recall_curve(y_test_, y_pred_naive_)
rf_precision, rf_recall, _ = precision_recall_curve(y_test, y_pred_rf_)
xg_precision, xg_recall, _ = precision_recall_curve(y_test, y_pred_xg)
# plot the precision-recall curves
fig=plt.figure(figsize=(10, 10))
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(ann_recall, ann_precision, marker='.', label='ANN')
plt.plot(lstm_recall, lstm_precision, marker='.', label='LSTM')
plt.plot(knn_recall, knn_precision, marker='.', label='KNN')
plt.plot(nb_recall, nb_precision, marker='.', label='Naive Bayes')
plt.plot(xg_recall, xg_precision, marker='.', label='Xg Boost')
plt.plot(rf_recall, rf_precision, marker='.', label='Random Forest')

# axis labels
fig.suptitle('Precision Recall Curve with ngram range(1,3) using k=7,a=10',fontsize=20)
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
fig.savefig('PR.png', dpi=fig.dpi)
files.download('PR.png') 
plt.show()

In [0]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Daraz_Distributed_Soft_Label_Final.xlsx',usecols=['Review', 'Label'])

y=data.iloc[:,1]
df_frequency_map={1:1,-1:0}
data.Label = data.Label.map(df_frequency_map)
data.head()

In [0]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=10000)
x = cv.fit_transform(corpus)
df1 = pd.DataFrame(x.toarray(), columns=cv.get_feature_names())

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df1)
scaled_data = scaler.transform(df1)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.10, random_state = 0)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 40,activation='sigmoid',input_dim = 20))
classifier.add(Dropout(0.3))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 20,activation='sigmoid'))
classifier.add(Dropout(0.3))
# Adding the output layer
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the ANN
optimizer = RMSprop(lr=0.01)
classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
'''model_history=classifier.fit(X_train, y_train,validation_split=0.1,batch_size = 128, nb_epoch = 30)

# list all data in history
print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

print(cm)
print(score)'''

In [0]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(X_train)
testing_set_scaled=sc.transform(X_test)

x_train = np.reshape(training_set_scaled, (training_set_scaled.shape[0], training_set_scaled.shape[1], 1))

x_test = np.reshape(testing_set_scaled, (testing_set_scaled.shape[0], testing_set_scaled.shape[1], 1))

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt

# Initialising the RNN
classifier1 = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
classifier1.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
classifier1.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
classifier1.add(LSTM(units = 50, return_sequences = True))
classifier1.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
classifier1.add(LSTM(units = 50, return_sequences = True))
classifier1.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
classifier1.add(LSTM(units = 50))
classifier1.add(Dropout(0.2))

# Adding the output layer
classifier1.add(Dense(units = 1))

# Compiling the RNN
classifier1.compile(optimizer = 'RMSprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the RNN to the Training set
model_history=classifier1.fit(x_train, y_train,validation_split=0.1,epochs = 10, batch_size = 128)

# list all data in history
print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predicting the Test set results
y_pred = classifier1.predict(x_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

print(score)
print(cm)


In [0]:
import matplotlib.pyplot as plt
from google.colab import files
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score,accuracy_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split

def plot_calibration_curve(est, name, fig_index):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=2, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=2, method='sigmoid')

    # Logistic regression with no calibration as baseline
    lr = LogisticRegression(C=1.)

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for clf, name in [(lr, 'Logistic'),
                      (est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        else:  # use decision function
            prob_pos = clf.decision_function(X_test)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        clf_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
        print("%s:" % name)
        print("\acc: %1.3f" % accuracy_score(y_test,y_pred))
        print("\tBrier: %1.3f" % (clf_score))
        print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))
        print("\tRecall: %1.3f" % recall_score(y_test, y_pred))
        print("\tF1: %1.3f\n" % f1_score(y_test, y_pred))

        fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=10)

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)
    fig.savefig('temp.png', dpi=fig.dpi)
    files.download("temp.png") 
    plt.tight_layout()


# Plot calibration curve for Linear SVC
plot_calibration_curve(LinearSVC(), "ANN", 2)

plt.show()
